In [1]:
import torch
import torchvision
import numpy as np
import gradient_reversal
import joblib
import os

In [2]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


# Import WIKI_EVE

In [3]:
import wiki_eve_max.model as models
import wiki_eve_max.loss as loss
import wiki_eve_max.transform as transform
import wiki_eve_max.dataset as dataset
import wiki_eve_max.work as work

In [4]:
CWD = os.getcwd()
META_FILE = os.path.join(CWD, "DATASETS_TEST/NP/even_spacing/mean/32", "meta.dump")
DATASET = dataset.BFIKEYSTROKE(
    meta_file=META_FILE,
    transform=torchvision.transforms.Compose([transform.Absolute()])
)

In [5]:
print(len(DATASET),len(DATASET.m_domains),DATASET.m_num_domain, DATASET.m_num_class)

140000 14000 14000 10


In [6]:
train_data, test_data = DATASET.split(train_p=0.75)
print(train_data[9])
print(train_data[25])

{'data': tensor([[0.3494, 0.5480, 0.1212,  ..., 0.1644, 0.7405, 0.2720],
        [0.3494, 0.5480, 0.1212,  ..., 0.1644, 0.7405, 0.2720],
        [0.3493, 0.5505, 0.1245,  ..., 0.1303, 0.7565, 0.2237],
        ...,
        [0.2152, 0.5077, 0.6731,  ..., 0.6474, 0.5685, 0.3081],
        [0.2603, 0.4807, 0.5138,  ..., 0.6273, 0.5158, 0.2983],
        [0.2603, 0.4807, 0.5138,  ..., 0.6273, 0.5158, 0.2983]]), 'label': 0, 'domain': 11898, 'domain_0': 8219}
{'data': tensor([[0.3369, 0.5474, 0.5558,  ..., 0.0569, 0.4816, 0.6091],
        [0.3409, 0.5413, 0.5451,  ..., 0.0961, 0.4366, 0.6298],
        [0.3377, 0.5487, 0.5507,  ..., 0.1102, 0.4276, 0.6136],
        ...,
        [0.6390, 0.5523, 0.4949,  ..., 0.4180, 0.5702, 0.7824],
        [0.6351, 0.5524, 0.4969,  ..., 0.4117, 0.5804, 0.7832],
        [0.6469, 0.5435, 0.4935,  ..., 0.4018, 0.5855, 0.7843]]), 'label': 7, 'domain': 921, 'domain_0': 12829}


In [8]:
learning_rate = 1e-3
batch_size = 32
epochs = 5

train_dataloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size)

model = models.WIKI_EVE_1D(
    conv_in_channels = 134,
    conv_out_channels = 75,
    conv_kernel_size = 12,
    conv_stride = 1,
    GF_output_size = 100,
    GC_labels = 10,
    GD_labels = DATASET.m_num_domain,
    lambda_domain = 0.01
)
print(model)
model.to(torch.device(device="cpu"))
loss_fn = loss.AdvLernLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    work.train_loop(train_dataloader, model, loss_fn, optimizer, batch_size=batch_size)#, device=device)
    work.test_loop(test_dataloader, model, loss_fn)#, device=device)
print("Done!")

WIKI_EVE_1D(
  (GF): Sequential(
    (0): Conv1d(134, 20, kernel_size=(12,), stride=(1,))
    (1): ReLU()
    (2): AdaptiveAvgPool1d(output_size=150)
    (3): Conv1d(20, 40, kernel_size=(12,), stride=(1,))
    (4): ReLU()
    (5): AdaptiveAvgPool1d(output_size=50)
  )
  (GC): Sequential(
    (0): Linear(in_features=2000, out_features=2000, bias=True)
    (1): ReLU()
    (2): Linear(in_features=2000, out_features=10, bias=True)
  )
  (GD): Sequential(
    (0): GradientReversal()
    (1): Linear(in_features=2000, out_features=2000, bias=True)
    (2): ReLU()
    (3): Linear(in_features=2000, out_features=14000, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 11.848948  [   32/105000]
